In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)


[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
dataset=stk_data

In [4]:
#column="Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.508982,0.441549,0.543815,0.491495
1,0.497006,0.433854,0.540997,0.513145
2,0.544411,0.479275,0.593970,0.563144
3,0.543912,0.504095,0.604114,0.552835
4,0.538423,0.489203,0.587489,0.576289
...,...,...,...,...
138,0.385230,0.330107,0.398422,0.404639
139,0.414421,0.355920,0.417864,0.401031
140,0.369261,0.296848,0.338405,0.316495
141,0.334331,0.252916,0.226543,0.275773


In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [10]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [15]:
def combination(data1, listt):
    # Print the list of variables being used
    print("Variables used:", listt)
    
    # Extract the subset of the dataset with the selected variables
    datasetTwo = dataset[listt]
    
    # Define the number of observations for testing
    test_obs = 28
    
    # Split the data into training and testing sets
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    # Import necessary libraries
    from statsmodels.tsa.statespace.varmax import VARMAX  
    from itertools import product
    
    # Initialize variables for storing best parameters
    best_aic = float('inf')
    best_params = None

    # Iterate over combinations of p and q values
    for p, q in product(range(1, 4), range(1, 4)):
        try:
            # Fit VARMAX model with current order
            model = VARMAX(train, order=(p, q))
            results = model.fit(maxiter=1000, disp=False)
            
            # Update best parameters if current AIC is lower
            aic = results.aic
            if aic < best_aic:
                best_aic = aic
                best_params = (p, q)
        except:
            continue

    # Print the best AIC and corresponding parameters
    print("Best AIC:", best_aic)
    print("Best Parameters (p, q):", best_params)

    # Fit VARMAX model with the best parameters
    model = VARMAX(train, order=best_params) 
    results = model.fit()
    
    # Forecast
    lagged_values = train.values[-max(best_params):]
    pred = results.forecast(y=lagged_values, steps=test_obs) 
    preds = pd.DataFrame(pred, columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    
    # Calculate performance metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
    rmse = round(mean_squared_error(test, pred, squared=False))
    mape = mean_absolute_percentage_error(test, pred)
    
    # Store performance metrics in a dictionary
    performance = {
        "Model": [listt],
        "RMSE": [rmse],
        "MaPe": [mape],
        "Lag": [best_params],
        "Test": [test_obs]
    }
    
    # Create a DataFrame with performance metrics and return
    perf = pd.DataFrame(performance)
    return perf, results, pred


In [ ]:
perf,result,pred= combination(data1, listt)

Variables used: ['Close', 'High', 'Open', 'Low']


C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency 

In [17]:
pred

,Close,High,Open,Low
115,710.654732,718.879764,710.068349,702.802862
116,712.450883,718.917720,712.329800,704.215302
117,713.915098,721.257857,713.706868,706.931370
118,715.958357,723.084726,715.298569,708.536611
119,717.942448,725.078117,717.327868,710.530247
120,719.840916,727.036856,719.305195,712.415955
121,721.645726,728.911349,721.200556,714.195274
122,723.352956,730.690948,723.002652,715.876693
123,724.964635,732.373682,724.707492,717.463085
124,726.484804,733.961984,726.317001,718.958969


In [14]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",33,0.039356,"(1, 2)",28
